In [ ]:
%load_ext autoreload
%autoreload 2

## Testing the Composites Discovery App with n=3 phases

In [ ]:
from hashin_shtrikman_mp.core.user_input import MaterialProperty, Material, MixtureProperty, Mixture, UserInput

# Define properties for each material
properties_mat_1 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=120, lower_bound=1e-7), # upper_bound=20, lower_bound=1 
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.0001, lower_bound=1e-5 
    MaterialProperty(prop='bulk_modulus', upper_bound=500, lower_bound=50),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=80),
    MaterialProperty(prop='universal_anisotropy', upper_bound=6, lower_bound=1),
]

properties_mat_2 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=5, lower_bound=2
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.009, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=400, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=4.3, lower_bound=1.3),
]

properties_mat_3 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=10, lower_bound=1
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.005, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=700, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=600, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=2.1, lower_bound=0.9),
]

# Define properties for the mixture
properties_mixture = [
    MixtureProperty(prop='elec_cond_300k_low_doping', desired_prop=9),
    MixtureProperty(prop='therm_cond_300k_low_doping', desired_prop=0.9),
    MixtureProperty(prop='bulk_modulus', desired_prop=280),
    MixtureProperty(prop='shear_modulus', desired_prop=230),
    MixtureProperty(prop='universal_anisotropy', desired_prop=1.5),
]

# Create Material & Mixture instances
mat_1 = Material(name='mat_1', properties=properties_mat_1)
mat_2 = Material(name='mat_2', properties=properties_mat_2)
mat_3 = Material(name='mat_3', properties=properties_mat_3)
mixture = Mixture(name='mixture', properties=properties_mixture)
aggregate = [mat_1, mat_2, mat_3, mixture]

# Initialize UserInput instance with materials and mixtures
user_input= UserInput(materials=[mat_1, mat_2, mat_3], mixtures=[mixture])
print("User Input: ", user_input)

# Initialize dictionaries to store the overall upper and lower bounds for each property
overall_bounds = {}

# Iterate over materials
for entity in aggregate:
    # Skip the mixture as it doesn't have upper and lower bounds
    if isinstance(entity, Material):
        for property in entity.properties:
            prop_name = property.prop

            # Initialize the overall bounds if they are not already present for the property
            if prop_name not in overall_bounds:
                overall_bounds[prop_name] = {'upper_bound': property.upper_bound, 'lower_bound': property.lower_bound}
            else:
                # Update overall upper and lower bounds by comparing with existing values
                overall_bounds[prop_name]['upper_bound'] = max(overall_bounds[prop_name]['upper_bound'], property.upper_bound)
                overall_bounds[prop_name]['lower_bound'] = min(overall_bounds[prop_name]['lower_bound'], property.lower_bound)

# Print the overall bounds for each property
print("Overall Upper & Lower Bounds:")
for prop, bounds in overall_bounds.items():
    print(f"Property: {prop}, Upper Bound: {bounds['upper_bound']}, Lower Bound: {bounds['lower_bound']}")

# Step 1: Create the consolidated_dict
overall_bounds_dict = {}
for prop, bounds in overall_bounds.items():
    overall_bounds_dict[prop] = {
        'upper_bound': bounds['upper_bound'],
        'lower_bound': bounds['lower_bound']
    }

print(overall_bounds_dict)

In [ ]:
from hashin_shtrikman_mp.core.optimizer import Optimizer
optimizer = Optimizer(api_key="uJpFxJJGKCSp9s1shwg9HmDuNjCDfWbM", user_input=user_input)

print("Property categories: ", optimizer.property_categories)
print("Property docs: ", optimizer.property_docs)

print("Lower Bounds:", optimizer.lower_bounds)
print("Upper Bounds:", optimizer.upper_bounds)

print("Number of Materials:", optimizer.num_materials)
print("Number of Properties:", optimizer.num_properties)

In [ ]:
# # Testing without calls to generate final dict (faster)
# import json
# consolidated_dict = {}
# with open("consolidated_dict_02_11_2024_23_45_58") as f:
#     consolidated_dict = json.load(f)
# print(consolidated_dict)

In [ ]:
# Run the optimization to obtain n=3 materials and their volume fractions that achieve the desired composite material properties
optimizer.set_HS_optim_params(gen_counter=True)

In [ ]:
# Create an instance of the Visualizer class using the instance of the Optimizer class
from hashin_shtrikman_mp.core.visualizer import Visualizer
visualizer = Visualizer(optimizer)

In [ ]:
# Print the optimization results as a table
visualizer.print_table_of_best_designs(rows=10)

In [ ]:
# Plot the genetic algorithm convergence plot
visualizer.plot_optimization_results()

In [ ]:
# Plot the contributions to the cost function for the best performer
visualizer.plot_cost_func_contribs()

In [ ]:
# Create an instance of the MatchFinder class using the instance of the Optimizer class
from hashin_shtrikman_mp.core.match_finder import MatchFinder
match_finder = MatchFinder(optimizer)

In [ ]:
# Get material matches
matches_dict = match_finder.get_material_matches(overall_bounds_dict)
print(f'Material Matches: {matches_dict}')

In [ ]:
import os
import json
from datetime import datetime

# Directory where the files are located
directory = './'

# Function to get the latest consolidated dict file
def get_latest_consolidated_dict_file(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Filter out only the consolidated_dict files
    consolidated_files = [f for f in files if f.startswith('consolidated_dict_')]
    
    # Sort the files by date and time in the filename
    consolidated_files.sort(key=lambda x: datetime.strptime(x, 'consolidated_dict_%m_%d_%Y_%H_%M_%S'))
    
    # Return the latest file
    latest_file = consolidated_files[-1] if consolidated_files else None
    return latest_file

# Get the latest consolidated dict file
latest_file = get_latest_consolidated_dict_file(directory)

if latest_file:
    # Open the latest consolidated dict file
    with open(os.path.join(directory, latest_file), 'r') as f:
        consolidated_dict = json.load(f)
        print(f"Loaded consolidated_dict from {latest_file}")
else:
    print("No consolidated dict file found.")


match_finder.get_material_match_costs(matches_dict=matches_dict, consolidated_dict=consolidated_dict)

In [ ]:
# For testing only
with open(f"test_consolidated_dict", encoding='utf-8') as f:
    test_consolidated_dict = json.load(f)

fake_matches_dict = match_finder.get_material_matches(consolidated_dict=test_consolidated_dict, threshold=1)
print(f'fake_matches_dict: {fake_matches_dict}')

match_finder.get_material_match_costs(matches_dict=fake_matches_dict, consolidated_dict=test_consolidated_dict)

In [ ]:
from mp_api.client import MPRester

# Base query initialization
query = {}

# Iterate over the properties in the overall_bounds_dict and dynamically build the query
for prop, bounds in overall_bounds_dict.items():
    # Skip 'elec_cond_300k_low_doping' and 'therm_cond_300k_low_doping'
    if prop in ['elec_cond_300k_low_doping', 'therm_cond_300k_low_doping']:
        continue  # Skip the current iteration

    # Proceed if 'upper_bound' and 'lower_bound' exist for the property
    if 'upper_bound' in bounds and 'lower_bound' in bounds:
        query[prop] = (bounds['lower_bound'], bounds['upper_bound'])

# Print the query before changes
#print(f"Initial query: {query}")

# Add additional fields you want to query, like 'material_id', 'formula_pretty', and all the properties in the initial query
fields = ['material_id', 'formula_pretty']  # Fixed fields
fields.extend(query.keys())  # Adding all the keys from the query to the fields list
print(f"Fields: {fields}")

# Change 'bulk_modulus' to 'k_voigt'
if 'bulk_modulus' in query:
    query['k_vrh'] = query.pop('bulk_modulus')

# Change 'shear_modulus' to 'g_voigt'
if 'shear_modulus' in query:
    query['g_vrh'] = query.pop('shear_modulus')

# Change 'universal_anisotropy' to 'elastic_anisotropy'
if 'universal_anisotropy' in query:
    query['elastic_anisotropy'] = query.pop('universal_anisotropy')

# change 'e_ij_max' to 'piezoelectric_modulus'
if 'e_ij_max' in query:
    query['piezoelectric_modulus'] = query.pop('e_ij_max')

mpr = MPRester("QePM93qZsMKNPkI4fEYaJfB7dONoQjaM")

print(f"Final query: {query}")

# Perform the query on the Materials Project database using the built query
materials = mpr.materials.summary.search(
    # k_vrh = (20.0, 700.0),
    # **query,  # Dynamically passing the property bounds as query filters
    # fields=fields,
    # num_chunks=100
)